https://www.gresearch.co.uk/blog/article/faster-string-processing-in-pandas/

https://gist.github.com/anshoomehra/ead8925ea291e233a5aa2dcaa2dc61b2

In [1]:
import pandas as pd
import cython
import os
import re
import json
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [2]:
os.chdir('/mnt/d/workspace/8-2/Financial-Statements-Text-Analysis/')

In [3]:
# params
with open('config.json', 'r') as f:
    c = json.load(f)
input_dir = os.path.join(c['DATA_DIR'], '10k_raw')
destination_dir = os.path.join(c['DATA_DIR'], '10k_clean')

In [4]:
metadata = pd.read_csv(os.path.join(c['DATA_DIR'], 'metadata.csv'))
metadata_legacy = pd.read_csv(os.path.join(c['DATA_DIR'], 'metadata_2017.csv'))

# only download the data from russell 3000 today
metadata = metadata_legacy[metadata_legacy['TICKER'].isin(metadata['ticker'])]

In [7]:
def parse_filing(text):
    regex_10k = re.search(r"(?s)(?m)<TYPE>10-K.*?(</TEXT>)", text)
    regex_items = re.compile(r"(> +Item|>Item|^Item|ITEM)((\s|&#160;|&nbsp;)(1A|1B|7A|7|8))")
    
    try: 
        extracted_text = regex_10k.group(0)
    except:
        print('❌ No 10-K match for file')
    
    item_matches = regex_items.finditer(extracted_text)
    matches_df = pd.DataFrame([(x.group(), x.start()) for x in item_matches])
    matches_df.columns = ['item', 'start']
    matches_df['item'] = matches_df['item'].str.lower().replace('>|\.| |&#160;|&nbsp|;', '', regex=True)
    matches_df = matches_df.sort_values('start').drop_duplicates(subset=['item'], keep='last')
    matches_df['end'] = matches_df['start'].shift(-1).fillna(0).astype(int)
    matches_df = matches_df[matches_df['item'].isin(['item1a', 'item1b', 'item7', 'item7a'])]
    if len(matches_df) != 4:
        print('❌ Only {sections} sections found for file'.format(sections=len(matches_df)))
    matches_df['text'] = matches_df\
        .apply(lambda row: BeautifulSoup(extracted_text[row['start']:row['end']], 'lxml').get_text('\n\n'), axis=1)
    
    return matches_df

def process_filing(i):
    ticker = metadata.iloc[i]['TICKER']
    leaf = metadata.iloc[i]['EDGAR_LINK'].split("/")[-1]
    
    input_file_name = os.path.join(input_dir,
                             ticker, leaf)
    with open(input_file_name, 'r', encoding='utf-8') as f:
        file = f.read()
    
    try:
        clean_df = parse_filing(file)
    
        destination_file = os.path.join(destination_dir, ticker, leaf)

        destination_subdir = os.path.dirname(destination_file)

        if not os.path.isdir(destination_subdir):
            os.makedirs(destination_subdir)

        clean_df.to_csv(destination_file)
        print('Successfully parsed for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        return clean_df
        
    except Exception as e:
        print('Failed parse for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        print(e)
        print('\n')

In [8]:
%load_ext line_profiler

In [8]:
%lprun -f parse_filing process_filing(10)

UsageError: Cell magic `%%lprun` not found (But line magic `%lprun` exists, did you mean that instead?).


In [10]:
%lprun -f process_filing process_filing(10)

Successfully parsed for file data/10k_raw/AEE/0001445305-13-000414.txt and ticker AEE


# Just doing it as a for loop

In [9]:
%%timeit
for i in range(10):
    ticker = metadata.iloc[i]['TICKER']
    leaf = metadata.iloc[i]['EDGAR_LINK'].split("/")[-1]
    
    input_file_name = os.path.join(input_dir,
                             ticker, leaf)
    with open(input_file_name, 'r', encoding='utf-8') as f:
        file = f.read()
    
    try:
        clean_df = parse_filing(file)
    
        destination_file = os.path.join(destination_dir, ticker, leaf)

        destination_subdir = os.path.dirname(destination_file)

        if not os.path.isdir(destination_subdir):
            os.makedirs(destination_subdir)

        clean_df.to_csv(destination_file)
        print('Successfully parsed for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        
    except Exception as e:
        print('Failed parse for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        print(e)
        print('\n')

Successfully parsed for file data/10k_raw/HSIC/0001000228-13-000010.txt and ticker HSIC
❌ Only 3 sections found for file
Successfully parsed for file data/10k_raw/NVAX/0001144204-13-014545.txt and ticker NVAX
Successfully parsed for file data/10k_raw/WAT/0001193125-13-076804.txt and ticker WAT
Successfully parsed for file data/10k_raw/NSP/0001140361-13-005779.txt and ticker NSP
Successfully parsed for file data/10k_raw/DISH/0001104659-13-011967.txt and ticker DISH
Successfully parsed for file data/10k_raw/SGMO/0001193125-13-076887.txt and ticker SGMO
❌ Only 2 sections found for file
Successfully parsed for file data/10k_raw/TGTX/0001144204-13-016691.txt and ticker TGTX
Successfully parsed for file data/10k_raw/NWPX/0001193125-13-112587.txt and ticker NWPX
Successfully parsed for file data/10k_raw/SCCO/0001104659-13-015720.txt and ticker SCCO
Successfully parsed for file data/10k_raw/INSG/0001144204-13-013543.txt and ticker INSG
Successfully parsed for file data/10k_raw/HSIC/0001000228-

# Multiprocessing with Pool

In [10]:
%%timeit
with Pool(processes=4) as pool:
    pool.map(process_filing, range(10))

❌ Only 3 sections found for file
Successfully parsed for file data/10k_raw/NVAX/0001144204-13-014545.txt and ticker NVAX
Successfully parsed for file data/10k_raw/WAT/0001193125-13-076804.txt and ticker WAT
Successfully parsed for file data/10k_raw/NSP/0001140361-13-005779.txt and ticker NSP
❌ Only 2 sections found for file
Successfully parsed for file data/10k_raw/TGTX/0001144204-13-016691.txt and ticker TGTX
Successfully parsed for file data/10k_raw/HSIC/0001000228-13-000010.txt and ticker HSIC
Successfully parsed for file data/10k_raw/SGMO/0001193125-13-076887.txt and ticker SGMO
Successfully parsed for file data/10k_raw/INSG/0001144204-13-013543.txt and ticker INSG
Successfully parsed for file data/10k_raw/NWPX/0001193125-13-112587.txt and ticker NWPX
Successfully parsed for file data/10k_raw/DISH/0001104659-13-011967.txt and ticker DISH
Successfully parsed for file data/10k_raw/SCCO/0001104659-13-015720.txt and ticker SCCO
❌ Only 3 sections found for file
Successfully parsed for f

# Cythonized

In [1]:
%load_ext Cython

In [10]:
%%cython

import pandas as pd
import cython
import os
import re
import json
from bs4 import BeautifulSoup
from multiprocessing import Pool

os.chdir('/mnt/d/workspace/8-2/Financial-Statements-Text-Analysis/')

# params
with open('config.json', 'r') as f:
    c = json.load(f)
input_dir = os.path.join(c['DATA_DIR'], '10k_raw')
destination_dir = os.path.join(c['DATA_DIR'], '10k_clean')

metadata = pd.read_csv(os.path.join(c['DATA_DIR'], 'metadata.csv'))
metadata_legacy = pd.read_csv(os.path.join(c['DATA_DIR'], 'metadata_2017.csv'))

# only download the data from russell 3000 today
metadata = metadata_legacy[metadata_legacy['TICKER'].isin(metadata['ticker'])]

def parse_filing(text):
    regex_10k = re.search(r"(?s)(?m)<TYPE>10-K.*?(</TEXT>)", text)
    regex_items = re.compile(r"(> +Item|>Item|^Item|ITEM)((\s|&#160;|&nbsp;)(1A|1B|7A|7|8))")
    
    try: 
        extracted_text = regex_10k.group(0)
    except:
        print('❌ No 10-K match for file')
    
    item_matches = regex_items.finditer(extracted_text)
    matches_df = pd.DataFrame([(x.group(), x.start()) for x in item_matches])
    matches_df.columns = ['item', 'start']
    matches_df['item'] = matches_df['item'].str.lower().replace('>|\.| |&#160;|&nbsp|;', '', regex=True)
    matches_df = matches_df.sort_values('start').drop_duplicates(subset=['item'], keep='last')
    matches_df['end'] = matches_df['start'].shift(-1).fillna(0).astype(int)
    matches_df = matches_df[matches_df['item'].isin(['item1a', 'item1b', 'item7', 'item7a'])]
    if len(matches_df) != 4:
        print('❌ Only {sections} sections found for file'.format(sections=len(matches_df)))
    matches_df['text'] = matches_df\
        .apply(lambda row: BeautifulSoup(extracted_text[row['start']:row['end']], 'lxml').get_text('\n\n'), axis=1)
    
    return matches_df

def process_filing(i):
    ticker = metadata.iloc[i]['TICKER']
    leaf = metadata.iloc[i]['EDGAR_LINK'].split("/")[-1]
    
    input_file_name = os.path.join(input_dir,
                             ticker, leaf)
    with open(input_file_name, 'r', encoding='utf-8') as f:
        file = f.read()
    
    try:
        clean_df = parse_filing(file)
    
        destination_file = os.path.join(destination_dir, ticker, leaf)

        destination_subdir = os.path.dirname(destination_file)

        if not os.path.isdir(destination_subdir):
            os.makedirs(destination_subdir)

        clean_df.to_csv(destination_file)
        print('Successfully parsed for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        
    except Exception as e:
        print('Failed parse for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        print(e)
        print('\n')

In [6]:
%%timeit 
with Pool(processes=4) as pool:
    pool.map(process_filing, range(10))

❌ Only 3 sections found for file
Successfully parsed for file data/10k_raw/NVAX/0001144204-13-014545.txt and ticker NVAX
Successfully parsed for file data/10k_raw/NSP/0001140361-13-005779.txt and ticker NSP
Successfully parsed for file data/10k_raw/WAT/0001193125-13-076804.txt and ticker WAT
Successfully parsed for file data/10k_raw/HSIC/0001000228-13-000010.txt and ticker HSIC
❌ Only 2 sections found for file
Successfully parsed for file data/10k_raw/SGMO/0001193125-13-076887.txt and ticker SGMO
Successfully parsed for file data/10k_raw/TGTX/0001144204-13-016691.txt and ticker TGTX
Successfully parsed for file data/10k_raw/NWPX/0001193125-13-112587.txt and ticker NWPX
Successfully parsed for file data/10k_raw/INSG/0001144204-13-013543.txt and ticker INSG
Successfully parsed for file data/10k_raw/DISH/0001104659-13-011967.txt and ticker DISH
Successfully parsed for file data/10k_raw/SCCO/0001104659-13-015720.txt and ticker SCCO
❌ Only 3 sections found for file
Successfully parsed for f

In [7]:
%%timeit
for i in range(10):
    ticker = metadata.iloc[i]['TICKER']
    leaf = metadata.iloc[i]['EDGAR_LINK'].split("/")[-1]
    
    input_file_name = os.path.join(input_dir,
                             ticker, leaf)
    with open(input_file_name, 'r', encoding='utf-8') as f:
        file = f.read()
    
    try:
        clean_df = parse_filing(file)
    
        destination_file = os.path.join(destination_dir, ticker, leaf)

        destination_subdir = os.path.dirname(destination_file)

        if not os.path.isdir(destination_subdir):
            os.makedirs(destination_subdir)

        clean_df.to_csv(destination_file)
        print('Successfully parsed for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        
    except Exception as e:
        print('Failed parse for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        print(e)
        print('\n')

Successfully parsed for file data/10k_raw/HSIC/0001000228-13-000010.txt and ticker HSIC
❌ Only 3 sections found for file
Successfully parsed for file data/10k_raw/NVAX/0001144204-13-014545.txt and ticker NVAX
Successfully parsed for file data/10k_raw/WAT/0001193125-13-076804.txt and ticker WAT
Successfully parsed for file data/10k_raw/NSP/0001140361-13-005779.txt and ticker NSP
Successfully parsed for file data/10k_raw/DISH/0001104659-13-011967.txt and ticker DISH
Successfully parsed for file data/10k_raw/SGMO/0001193125-13-076887.txt and ticker SGMO
❌ Only 2 sections found for file
Successfully parsed for file data/10k_raw/TGTX/0001144204-13-016691.txt and ticker TGTX
Successfully parsed for file data/10k_raw/NWPX/0001193125-13-112587.txt and ticker NWPX
Successfully parsed for file data/10k_raw/SCCO/0001104659-13-015720.txt and ticker SCCO
Successfully parsed for file data/10k_raw/INSG/0001144204-13-013543.txt and ticker INSG
Successfully parsed for file data/10k_raw/HSIC/0001000228-

# Cython with declarations

In [17]:
%%cython

import pandas as pd
import cython
import os
import re
import json
from bs4 import BeautifulSoup
from multiprocessing import Pool

os.chdir('/mnt/d/workspace/8-2/Financial-Statements-Text-Analysis/')

# params
with open('config.json', 'r') as f:
    c = json.load(f)
input_dir = os.path.join(c['DATA_DIR'], '10k_raw')
destination_dir = os.path.join(c['DATA_DIR'], '10k_clean')

metadata = pd.read_csv(os.path.join(c['DATA_DIR'], 'metadata.csv'))
metadata_legacy = pd.read_csv(os.path.join(c['DATA_DIR'], 'metadata_2017.csv'))

# only download the data from russell 3000 today
metadata = metadata_legacy[metadata_legacy['TICKER'].isin(metadata['ticker'])]

@cython.boundscheck(False)
@cython.wraparound(False)
def parse_filing_cython(text):
    cdef str extracted_text
    cdef int start, end, sections
    cdef Py_ssize_t i, n
    
    regex_10k = re.compile(r"(?s)(?m)<TYPE>10-K.*?(</TEXT>)")
    regex_items = re.compile(r"(> +Item|>Item|^Item|ITEM)((\s|&#160;|&nbsp;)(1A|1B|7A|7|8))")
    
    try:
        extracted_text = regex_10k.search(text).group(0)
    except:
        print('❌ No 10-K match for file')
    
    item_matches = regex_items.finditer(extracted_text)
    matches_df = pd.DataFrame([(x.group(), x.start()) for x in item_matches])
    matches_df.columns = ['item', 'start']
    matches_df['item'] = matches_df['item'].str.lower().replace('>|\.| |&#160;|&nbsp|;', '', regex=True)
    matches_df = matches_df.sort_values('start').drop_duplicates(subset=['item'], keep='last')
    matches_df['end'] = matches_df['start'].shift(-1).fillna(0).astype(int)
    matches_df = matches_df[matches_df['item'].isin(['item1a', 'item1b', 'item7', 'item7a'])]
    n = len(matches_df)
    if n != 4:
        print('❌ Only {sections} sections found for file'.format(sections=n))
    for i in range(n):
        start = matches_df.iloc[i]['start']
        end = matches_df.iloc[i+1]['start'] if i+1<n else 0
        matches_df.at[i, 'text'] = BeautifulSoup(extracted_text[start:end], 'lxml').get_text('\n\n')
    
    return matches_df

def process_filing(i):
    ticker = metadata.iloc[i]['TICKER']
    leaf = metadata.iloc[i]['EDGAR_LINK'].split("/")[-1]
    
    input_file_name = os.path.join(input_dir,
                             ticker, leaf)
    with open(input_file_name, 'r', encoding='utf-8') as f:
        file = f.read()
    
    try:
        clean_df = parse_filing_cython(file)
    
        destination_file = os.path.join(destination_dir, ticker, leaf)

        destination_subdir = os.path.dirname(destination_file)

        if not os.path.isdir(destination_subdir):
            os.makedirs(destination_subdir)

        clean_df.to_csv(destination_file)
        print('Successfully parsed for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        
    except Exception as e:
        print('Failed parse for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        print(e)
        print('\n')
        


In [16]:
%%timeit
with Pool(processes=4) as pool:
    pool.map(process_filing, range(10))

❌ Only 3 sections found for file
Successfully parsed for file data/10k_raw/NVAX/0001144204-13-014545.txt and ticker NVAX
Successfully parsed for file data/10k_raw/NSP/0001140361-13-005779.txt and ticker NSP
Successfully parsed for file data/10k_raw/WAT/0001193125-13-076804.txt and ticker WAT
Successfully parsed for file data/10k_raw/HSIC/0001000228-13-000010.txt and ticker HSIC
❌ Only 2 sections found for file
Successfully parsed for file data/10k_raw/SGMO/0001193125-13-076887.txt and ticker SGMO
Successfully parsed for file data/10k_raw/TGTX/0001144204-13-016691.txt and ticker TGTX
Successfully parsed for file data/10k_raw/NWPX/0001193125-13-112587.txt and ticker NWPX
Successfully parsed for file data/10k_raw/INSG/0001144204-13-013543.txt and ticker INSG
Successfully parsed for file data/10k_raw/DISH/0001104659-13-011967.txt and ticker DISH
Successfully parsed for file data/10k_raw/SCCO/0001104659-13-015720.txt and ticker SCCO
❌ Only 3 sections found for file
Successfully parsed for f

# Batch all files to process

In [ ]:
for i in range(len(metadata)):
    ticker = metadata.iloc[i]['TICKER']
    leaf = metadata.iloc[i]['EDGAR_LINK'].split("/")[-1]
    
    input_file_name = os.path.join(input_dir,
                             ticker, leaf)
    with open(input_file_name, 'r', encoding='utf-8') as f:
        file = f.read()
    
    try:
        clean_df = parse_filing(file)
    
        destination_file = os.path.join(destination_dir, ticker, leaf)

        destination_subdir = os.path.dirname(destination_file)

        if not os.path.isdir(destination_subdir):
            os.makedirs(destination_subdir)

        clean_df.to_csv(destination_file, index=False)
        print('Successfully parsed for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        
    except Exception as e:
        print('Failed parse for file {} and ticker {}'.format(input_file_name, metadata.iloc[i]['TICKER']))
        print(e)
        print('\n')

Successfully parsed for file data/10k_raw/HSIC/0001000228-13-000010.txt and ticker HSIC
❌ Only 3 sections found for file
Successfully parsed for file data/10k_raw/NVAX/0001144204-13-014545.txt and ticker NVAX
Successfully parsed for file data/10k_raw/WAT/0001193125-13-076804.txt and ticker WAT
Successfully parsed for file data/10k_raw/NSP/0001140361-13-005779.txt and ticker NSP
Successfully parsed for file data/10k_raw/DISH/0001104659-13-011967.txt and ticker DISH
Successfully parsed for file data/10k_raw/SGMO/0001193125-13-076887.txt and ticker SGMO
❌ Only 2 sections found for file
Successfully parsed for file data/10k_raw/TGTX/0001144204-13-016691.txt and ticker TGTX
Successfully parsed for file data/10k_raw/NWPX/0001193125-13-112587.txt and ticker NWPX
Successfully parsed for file data/10k_raw/SCCO/0001104659-13-015720.txt and ticker SCCO
Successfully parsed for file data/10k_raw/INSG/0001144204-13-013543.txt and ticker INSG
Successfully parsed for file data/10k_raw/AEE/0001445305-1

Successfully parsed for file data/10k_raw/MBWM/0001193125-13-106327.txt and ticker MBWM
Successfully parsed for file data/10k_raw/PDM/0001042776-13-000009.txt and ticker PDM
Successfully parsed for file data/10k_raw/DRQ/0001193125-13-081240.txt and ticker DRQ
Successfully parsed for file data/10k_raw/NLY/0001157523-13-001038.txt and ticker NLY
Successfully parsed for file data/10k_raw/CHRW/0001043277-13-000004.txt and ticker CHRW
Successfully parsed for file data/10k_raw/SRI/0001144204-13-014041.txt and ticker SRI
Successfully parsed for file data/10k_raw/SAH/0001193125-13-127041.txt and ticker SAH
Successfully parsed for file data/10k_raw/JNPR/0001043604-13-000022.txt and ticker JNPR
Successfully parsed for file data/10k_raw/EPR/0001045450-13-000019.txt and ticker EPR
Successfully parsed for file data/10k_raw/BAX/0001193125-13-069609.txt and ticker BAX
Successfully parsed for file data/10k_raw/NVDA/0001045810-13-000008.txt and ticker NVDA
Successfully parsed for file data/10k_raw/HFWA

Successfully parsed for file data/10k_raw/LAMR/0001193125-13-084458.txt and ticker LAMR
Successfully parsed for file data/10k_raw/UPS/0001090727-13-000005.txt and ticker UPS
Successfully parsed for file data/10k_raw/CHTR/0001091667-13-000020.txt and ticker CHTR
Successfully parsed for file data/10k_raw/CIR/0001091883-13-000005.txt and ticker CIR
Successfully parsed for file data/10k_raw/WWE/0001445305-13-000441.txt and ticker WWE
Successfully parsed for file data/10k_raw/SPSC/0001193125-13-093885.txt and ticker SPSC
Successfully parsed for file data/10k_raw/DXCM/0001193125-13-069590.txt and ticker DXCM
Successfully parsed for file data/10k_raw/ZION/0000109380-13-000081.txt and ticker ZION
Successfully parsed for file data/10k_raw/TDY/0001094285-13-000064.txt and ticker TDY
Successfully parsed for file data/10k_raw/BGCP/0001193125-13-101758.txt and ticker BGCP
Successfully parsed for file data/10k_raw/RE/0001095073-13-000015.txt and ticker RE
Successfully parsed for file data/10k_raw/HS

Successfully parsed for file data/10k_raw/VRNT/0001166388-13-000056.txt and ticker VRNT
Successfully parsed for file data/10k_raw/CMCSA/0001193125-13-067658.txt and ticker CMCSA
Successfully parsed for file data/10k_raw/WTBA/0001166928-13-000019.txt and ticker WTBA
Successfully parsed for file data/10k_raw/NE/0001193125-13-074360.txt and ticker NE
Successfully parsed for file data/10k_raw/CPSI/0001193125-13-101203.txt and ticker CPSI
Successfully parsed for file data/10k_raw/BANC/0001193125-13-132678.txt and ticker BANC
Successfully parsed for file data/10k_raw/DCT/0001193125-13-069475.txt and ticker DCT
Successfully parsed for file data/10k_raw/SAFT/0001047469-13-002988.txt and ticker SAFT
Successfully parsed for file data/10k_raw/HA/0001047469-13-000781.txt and ticker HA
Successfully parsed for file data/10k_raw/CEVA/0001193125-13-109922.txt and ticker CEVA
Successfully parsed for file data/10k_raw/HY/0001173514-13-000022.txt and ticker HY
❌ Only 1 sections found for file
Successfull

❌ Only 2 sections found for file
Successfully parsed for file data/10k_raw/DUK/0001326160-13-000009.txt and ticker DUK
Successfully parsed for file data/10k_raw/WDAY/0001193125-13-122351.txt and ticker WDAY
Successfully parsed for file data/10k_raw/BCC/0001328581-13-000015.txt and ticker BCC
Successfully parsed for file data/10k_raw/HOMB/0001193125-13-089830.txt and ticker HOMB
Successfully parsed for file data/10k_raw/BKD/0001332349-13-000012.txt and ticker BKD
Successfully parsed for file data/10k_raw/CROX/0001193125-13-072829.txt and ticker CROX
Successfully parsed for file data/10k_raw/Z/0001193125-13-071010.txt and ticker Z
Successfully parsed for file data/10k_raw/CCO/0001334978-13-000006.txt and ticker CCO
Successfully parsed for file data/10k_raw/LYV/0001193125-13-077102.txt and ticker LYV
❌ Only 3 sections found for file
Successfully parsed for file data/10k_raw/APP/0001336545-13-000009.txt and ticker APP
Successfully parsed for file data/10k_raw/LDOS/0001193125-13-127240.txt 

...

Successfully parsed for file data/10k_raw/SASR/0001144204-13-015676.txt and ticker SASR
Successfully parsed for file data/10k_raw/DELL/0000826083-13-000005.txt and ticker DELL
Successfully parsed for file data/10k_raw/ORRF/0001193125-13-093432.txt and ticker ORRF
Successfully parsed for file data/10k_raw/DX/0000826675-13-000006.txt and ticker DX
Successfully parsed for file data/10k_raw/EIX/0000827052-13-000024.txt and ticker EIX
Successfully parsed for file data/10k_raw/RBC/0000082811-13-000008.txt and ticker RBC
Successfully parsed for file data/10k_raw/WSFS/0001193125-13-112559.txt and ticker WSFS
Failed parse for file data/10k_raw/C/0001206774-13-000852.txt and ticker C
Length mismatch: Expected axis has 0 elements, new values have 2 elements


❌ Only 2 sections found for file
Successfully parsed for file data/10k_raw/FCX/0000831259-13-000014.txt and ticker FCX
Successfully parsed for file data/10k_raw/IEX/0001193125-13-069436.txt and ticker IEX
Successfully parsed for file data/10

Successfully parsed for file data/10k_raw/ICUI/0000883984-13-000016.txt and ticker ICUI
Successfully parsed for file data/10k_raw/VVI/0001193125-13-101356.txt and ticker VVI
Successfully parsed for file data/10k_raw/OFIX/0001193125-13-087561.txt and ticker OFIX
Successfully parsed for file data/10k_raw/ACT/0001193125-13-082059.txt and ticker ACT
Successfully parsed for file data/10k_raw/RCL/0001047469-13-001567.txt and ticker RCL
Successfully parsed for file data/10k_raw/CACC/0000885550-13-000018.txt and ticker CACC
Successfully parsed for file data/10k_raw/KSS/0000885639-13-000004.txt and ticker KSS
Successfully parsed for file data/10k_raw/BSX/0000885725-13-000007.txt and ticker BSX
Successfully parsed for file data/10k_raw/USPH/0001193125-13-102877.txt and ticker USPH
Successfully parsed for file data/10k_raw/FCEL/0000886128-13-000002.txt and ticker FCEL
Successfully parsed for file data/10k_raw/LGND/0000886163-13-000025.txt and ticker LGND
Successfully parsed for file data/10k_raw/

Successfully parsed for file data/10k_raw/EMN/0000915389-13-000010.txt and ticker EMN
Successfully parsed for file data/10k_raw/KEY/0001193125-13-076993.txt and ticker KEY
❌ Only 1 sections found for file
Successfully parsed for file data/10k_raw/AVB/0001047469-13-001516.txt and ticker AVB
Successfully parsed for file data/10k_raw/ALB/0001193125-13-062728.txt and ticker ALB
Successfully parsed for file data/10k_raw/MLM/0001193125-13-070711.txt and ticker MLM
❌ Only 3 sections found for file
Successfully parsed for file data/10k_raw/TSCO/0000916365-13-000006.txt and ticker TSCO
Successfully parsed for file data/10k_raw/DAR/0000916540-13-000006.txt and ticker DAR
Failed parse for file data/10k_raw/ADC/0001144204-13-014094.txt and ticker ADC
Length mismatch: Expected axis has 0 elements, new values have 2 elements


Successfully parsed for file data/10k_raw/RMBS/0000917273-13-000009.txt and ticker RMBS
Successfully parsed for file data/10k_raw/FARO/0001193125-13-080082.txt and ticker FARO

Successfully parsed for file data/10k_raw/TGI/0001021162-13-000018.txt and ticker TGI
Successfully parsed for file data/10k_raw/PLUS/0001022408-13-000025.txt and ticker PLUS
Successfully parsed for file data/10k_raw/EGHT/0001136261-13-000259.txt and ticker EGHT
Successfully parsed for file data/10k_raw/BFLY/0001133796-13-000038.txt and ticker BFLY
Successfully parsed for file data/10k_raw/RL/0001037038-13-000009.txt and ticker RL
Successfully parsed for file data/10k_raw/PETS/0001188112-13-001701.txt and ticker PETS
Successfully parsed for file data/10k_raw/SCS/0001050825-13-000083.txt and ticker SCS
Successfully parsed for file data/10k_raw/FBP/0001193125-13-136825.txt and ticker FBP
Successfully parsed for file data/10k_raw/NTCT/0001193125-13-235038.txt and ticker NTCT
Successfully parsed for file data/10k_raw/AMEH/0001144204-13-025748.txt and ticker AMEH
❌ Only 2 sections found for file
Successfully parsed for file data/10k_raw/WRLD/0000108385-13-000022.txt and ticker WRLD
Successfu

Successfully parsed for file data/10k_raw/URBN/0001193125-13-136738.txt and ticker URBN
Successfully parsed for file data/10k_raw/SJM/0001193125-13-266625.txt and ticker SJM
Successfully parsed for file data/10k_raw/HELE/0001104659-13-034243.txt and ticker HELE
Successfully parsed for file data/10k_raw/EXP/0001193125-13-235755.txt and ticker EXP
Successfully parsed for file data/10k_raw/MCK/0001561787-13-000010.txt and ticker MCK
Successfully parsed for file data/10k_raw/TTWO/0001047469-13-006006.txt and ticker TTWO
Successfully parsed for file data/10k_raw/TDW/0001193125-13-229622.txt and ticker TDW
Successfully parsed for file data/10k_raw/TRNS/0001171843-13-002603.txt and ticker TRNS
Successfully parsed for file data/10k_raw/EL/0001104659-13-065820.txt and ticker EL
Successfully parsed for file data/10k_raw/COTY/0000930413-13-004596.txt and ticker COTY
Successfully parsed for file data/10k_raw/OSIS/0001047469-13-008515.txt and ticker OSIS
Successfully parsed for file data/10k_raw/FD